# Cargamos módulos

In [1]:
import pandas as pd
import numpy as np
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px
import unicodedata

# Datos de Consumo de agua CDMX primer semestre 2019

In [ ]:
df = pd.read_csv("/home/jfractal/Documentos/dataton_2021/dataton_AGUA_2021/consumo-agua.csv")
#df.dtypes ## muestra los nombres de los campos y su tipo.
df.columns
#df.head()

In [5]:
df2 = df.iloc[:, [8,9,7,17,1,14,16]]
df2.head(1)
#print(df['bimestre'].unique()) ##muestra los valores únicos del campo bimestre

,alcaldia,colonia,consumo_total_dom,indice_des,geo_point_2d,bimestre,gid
0,GUSTAVO A. MADERO,7 DE NOVIEMBRE,468.23,ALTO,"19.4552601937,-99.1126617526",3,57250


In [ ]:
df2['lat'], df2['lon'] = df2['geo_point_2d'].str.split(',', 1).str ## Separamos el campo geo_point_2d en dos nuevas columnas
df2.head(20)

## Consumo total domiciliario por alcaldía

In [ ]:
consumoAguaAlcaldia = pd.DataFrame(df2.groupby(["alcaldia"]).consumo_total_dom.sum())##Agrupamos por alcaldía y calculamos su total de consumo

consumoAguaAlcaldia = consumoAguaAlcaldia.rename_axis('alcaldia').reset_index()
print(consumoAguaAlcaldia)


## Consumo total por colonia 

In [18]:
consumoAguaColonia = pd.DataFrame(df2.groupby(["colonia"]).consumo_total_dom.sum())##Agrupamos por colonia y calculamos su consumo Total de agua
consumoAguaColonia = consumoAguaColonia.rename_axis('colonia').reset_index()
print(consumoAguaColonia)


                   colonia  consumo_total_dom
0              10 DE ABRIL            3518.36
1               10 DE MAYO           39893.07
2          12 DE DICIEMBRE           14665.91
3             15 DE AGOSTO           22758.02
4         16 DE SEPTIEMBRE           26090.27
...                    ...                ...
1335         ZENON DELGADO            4709.78
1336            ZENTLAPATL              12.20
1337           ZONA CENTRO          258997.92
1338          ZONA ESCOLAR           78071.28
1339  ZONA ESCOLAR ORIENTE          162172.81

[1340 rows x 2 columns]


In [19]:
coordenasGeoColonia = df2.iloc[:,[1,4]]
consumoAguaColonia = consumoAguaColonia.merge(coordenasGeoColonia, left_on='colonia', right_on='colonia')


In [20]:
consumoAguaColonia2 = consumoAguaColonia.iloc[:,[0,1,2]]
consumoAguaColonia2 = consumoAguaColonia2.drop_duplicates(subset=['colonia'])
consumoAguaColonia2['lat'], consumoAguaColonia2['lon'] = consumoAguaColonia2['geo_point_2d'].str.split(',', 1).str
consumoAguaColonia2[['lat', 'lon']] = consumoAguaColonia2[['lat', 'lon']].astype(float)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: FutureWarning:

Columnar iteration over characters will be deprecated in future releases.



In [ ]:
consumoAguaIndiceDes = pd.DataFrame( ((df2.groupby(['indice_des']).consumo_total_dom.sum() )))##Agrupamos por índice de desarrollo y calculamos su consumo total de agua
consumoAguaIndiceDes.head()

In [ ]:
coloniasIndiceDes = pd.DataFrame(df2.indice_des.value_counts() )### Calculamos una tabla de frecuencias de colonias con su indice de desarrollo
print(coloniasIndiceDes)

# Datos de Población CDMX 2020

In [ ]:
pob = pd.read_csv("/home/jfractal/Documentos/dataton_2021/dataton_AGUA_2021/ITER2020_09CDMX.csv")
pob.columns

In [12]:
pob.columns
pobT = pob.iloc[:,[3,5,6]]
pobT = pobT.rename(columns={"Nombre del municipio o demarcación territorial": "NomMun", "Nombre de la localidad":"NomLoc", "Población total":"PT"  } )
pobT = pobT[ (pobT['NomLoc'] == "Total del Municipio") ]
pobT = pobT.iloc[:,[0,2]]
print(pobT)

                     NomMun         PT
3              Azcapotzalco   432205.0
5                  Coyoacán   614447.0
7     Cuajimalpa de Morelos   217686.0
34        Gustavo A. Madero  1173351.0
36                Iztacalco   404695.0
38               Iztapalapa  1835486.0
40   La Magdalena Contreras   247622.0
66               Milpa Alta   152685.0
313          Álvaro Obregón   759137.0
322                 Tláhuac   392313.0
377                 Tlalpan   699928.0
544              Xochimilco   442178.0
658           Benito Juárez   434153.0
660              Cuauhtémoc   545884.0
662          Miguel Hidalgo   414470.0
664     Venustiano Carranza   443704.0


# Creamos un dataset de la unión de datos de consumo de agua y población total de la cdmx

In [ ]:
#Homologamos el campo "alcaldia". Transformamos de minusculas a mayusculas y quitamos tíldes
pobT["NomMun"] = pobT["NomMun"].str.upper()##Tranformar de minusculas a mayusculas
pobT['NomMun']=pobT['NomMun'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')##Quita acentos
pobT = pobT.replace({"CUAJIMALPA DE MORELOS": "CUAJIMALPA",
               "LA MAGDALENA CONTRERAS": "MAGDALENA CONTRERAS" })##Remplazamos los registros
print(pobT)

In [ ]:
pobT_consAgua = pobT.merge(consumoAguaAlcaldia, left_on='NomMun', right_on='alcaldia')
#pobT_consAgua = pobT_consAgua.iloc[:,[0,1,3]]
#print(pobT_consAgua)
pobT.head(5)

In [ ]:
## Creamos nuevas variables

pobT_consAgua["ConsumoPercapita"] = pobT_consAgua.consumo_total_dom/pobT_consAgua.PT
pobT_consAgua.sort_values(by=['ConsumoPercapita'], inplace=True, ascending=False )
print(pobT_consAgua)

# Gráficas

## Gráfica de Consumo de agua por alcaldía

In [16]:
data = [go.Bar(x=consumoAguaAlcaldia["alcaldia"] , y=consumoAguaAlcaldia["consumo_total_dom"]   )]
layout = go.Layout(title="Consumo de Agua Domiciliario por Alcaldía\nCDMX 2019",
                    xaxis= dict(title="Alcaldías"),
                    yaxis= dict(title="Consumo Total de Agua"),
                    barmode = "stack" )
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig)

'temp-plot.html'

## Gráfica de la Población total por alcaldía

In [39]:
data1 = [go.Bar(x=pobT["NomMun"], y=pobT["PT"] )]
layout1 = go.Layout(title="Población Total de la CDMX por Alcaldía 2020", 
                    xaxis=dict(title="Alcaldía"),
                    yaxis=dict(title="Población Total"),
                    barmode="stack" )
fig1 = go.Figure(data=data1, layout=layout1)
pyo.plot(fig1, filename="poblacion_total.html")

'poblacion_total.html'

## Gráfica de Consumo de agua percapita

In [73]:
data2 = [go.Bar(x=pobT_consAgua["NomMun"], y=pobT_consAgua["ConsumoPercapita"])]
layout2 = go.Layout(title="Consumo per cápita de Agua en la CDMX 2019",
                     xaxis=dict(title="Alcaldía"),
                     yaxis=dict(title="Consumo de agua"),
                     barmode="stack")
fig2 = go.Figure(data=data2, layout=layout2)
pyo.plot(fig2, filename="consumo_agua_percapita.html")

'consumo_agua_percapita.html'

## Mapa de consumo de agua domiciliario por colonia

In [21]:
mapbox_access_token = "pk.eyJ1IjoiaXZhbmxvc2FyIiwiYSI6ImNrZTJpdWN0NDA5cXUyem1oOGx3NGh1bGsifQ.wuhB2vmk4QGrciFWYygqaA"

fig3 = px.scatter_mapbox(consumoAguaColonia2, lat = "lat", lon = "lon", hover_name= "colonia", 
                        hover_data = ["consumo_total_dom"],
                        color="consumo_total_dom", size_max = 15, zoom = 10,
                        size="consumo_total_dom",color_continuous_scale=px.colors.cyclical.IceFire,
                        title = "Consumo de agua por colonia", 
                        
                        )
fig3.update_layout(
    autosize=True,
    hovermode='closest', #muestra el dato más cercano al mover el cursor por el gráfico
    mapbox=dict(
        accesstoken=mapbox_access_token,
        zoom=12 #zoom inicial del mapa
    ),
)


pyo.plot(fig3, filename="ConsumoAguaDom.html")



'ConsumoAguaDom.html'